# LAB | Feature Engineering

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In [30]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsRegressor

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


**Check the shape of your data**

In [3]:
spaceship.shape

(8693, 14)

**Check for data types**

In [4]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

**Check for missing values**

In [5]:
spaceship.isnull().sum()/len(spaceship)*100

PassengerId     0.000000
HomePlanet      2.312205
CryoSleep       2.496261
Cabin           2.289198
Destination     2.093639
Age             2.059128
VIP             2.335212
RoomService     2.082135
FoodCourt       2.105142
ShoppingMall    2.392730
Spa             2.105142
VRDeck          2.162660
Name            2.300702
Transported     0.000000
dtype: float64

There are multiple strategies to handle missing data

- Removing all rows or all columns containing missing data.
- Filling all missing values with a value (mean in continouos or mode in categorical for example).
- Filling all missing values with an algorithm.

For this exercise, because we have such low amount of null values, we will drop rows containing any missing value. 

In [6]:
spaceship.dropna(inplace=True)

- **Cabin** is too granular - transform it in order to obtain {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}

In [7]:
spaceship.nunique()

PassengerId     6606
HomePlanet         3
CryoSleep          2
Cabin           5305
Destination        3
Age               80
VIP                2
RoomService     1093
FoodCourt       1293
ShoppingMall     984
Spa             1143
VRDeck          1106
Name            6590
Transported        2
dtype: int64

In [8]:
# Convert categorical values to numerical codes
codes, labels = pd.factorize(spaceship['Cabin'])
spaceship['Cabin_Code'] = codes

# Use pd.cut() on the numerical codes
series = pd.cut(spaceship['Cabin_Code'], 8, labels=['A','B','C','D','E','F','G','T'])
display(series.value_counts())

Cabin_Code
F    861
T    838
A    836
D    823
B    820
E    820
C    818
G    790
Name: count, dtype: int64

- Drop PassengerId and Name

In [9]:
spaceship.drop(columns=["PassengerId", "Name"], inplace=True)

- For non-numerical columns, do dummies.

In [10]:
spaceship_transformed = pd.get_dummies(spaceship, columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP'], drop_first=True)
spaceship_transformed.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin_Code,HomePlanet_Europa,HomePlanet_Mars,...,Cabin_G/994/S,Cabin_G/996/S,Cabin_G/998/S,Cabin_G/999/P,Cabin_G/999/S,Cabin_T/1/P,Cabin_T/3/P,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,True,False,...,False,False,False,False,False,False,False,False,True,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,1,False,False,...,False,False,False,False,False,False,False,False,True,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,2,True,False,...,False,False,False,False,False,False,False,False,True,True
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,2,True,False,...,False,False,False,False,False,False,False,False,True,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,3,False,False,...,False,False,False,False,False,False,False,False,True,False


**Perform Train Test Split**

In [11]:
features = spaceship_transformed.drop(columns=["Transported"])
target = spaceship_transformed["Transported"]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

**Model Selection**

In this exercise we will be using **KNN** as our predictive model.

In [23]:
normalizer = MinMaxScaler()

In [24]:
normalizer.fit(X_train)

MinMaxScaler()

In [25]:
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [26]:
X_train_norm

array([[4.05063291e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [5.06329114e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [3.79746835e-01, 0.00000000e+00, 7.91600979e-03, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       ...,
       [4.55696203e-01, 0.00000000e+00, 1.59527723e-01, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [4.30379747e-01, 0.00000000e+00, 1.34169658e-04, ...,
        0.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.77215190e-01, 2.01612903e-04, 2.95508671e-02, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [27]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_Code,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,...,Cabin_G/994/S,Cabin_G/996/S,Cabin_G/998/S,Cabin_G/999/P,Cabin_G/999/S,Cabin_T/1/P,Cabin_T/3/P,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,0.405063,0.00000,0.000000,0.000000,0.000000,0.000000,0.393174,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.050633,0.00000,0.000000,0.000000,0.000000,0.000000,0.839336,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.379747,0.00000,0.007916,0.000000,0.051276,0.000000,0.230436,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.215190,0.00131,0.000000,0.046111,0.016378,0.000049,0.574392,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.329114,0.00000,0.000000,0.000000,0.000000,0.000000,0.658118,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [28]:
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)
X_test_norm.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_Code,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,...,Cabin_G/994/S,Cabin_G/996/S,Cabin_G/998/S,Cabin_G/999/P,Cabin_G/999/S,Cabin_T/1/P,Cabin_T/3/P,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,0.632911,0.0,0.000000,0.0,0.00000,0.0,0.276447,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.227848,0.0,0.000000,0.0,0.00000,0.0,0.149538,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.189873,0.0,0.000000,0.0,0.00000,0.0,0.952668,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.658228,0.0,0.000000,0.0,0.00000,0.0,0.584009,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.784810,0.0,0.054775,0.0,0.07774,0.0,0.500094,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [31]:
knn = KNeighborsRegressor(n_neighbors=10)

In [32]:
knn.fit(X_train_norm, y_train)

KNeighborsRegressor(n_neighbors=10)

- Evaluate your model's performance. Comment it

In [33]:
knn.score(X_test_norm, y_test)

0.3223298033282904

In [34]:
pred = knn.predict(X_test_norm)

In [35]:
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_test, pred)
RMSE = mean_squared_error(y_test, pred, squared=False)

print(MSE)
print(RMSE)

0.1694175491679274
0.4116036311403574


- The Mean Squared Error (MSE) is approximately 0.1694. This indicates the average squared difference between the actual and predicted values. A lower MSE suggests better model performance, with 0 being the ideal value. An MSE of 0.1694 indicates that, on average, the squared difference between the actual and predicted values is around 0.1694.

- The Root Mean Squared Error (RMSE) is approximately 0.4116. It is the square root of the MSE and provides a measure of the average magnitude of error between the actual and predicted values. A lower RMSE indicates better model performance, with 0 being the ideal value. The RMSE is 0.4116, indicating that, on average, the difference between the actual and predicted values is around 0.6264

These error metrics suggest that the model's performance is reasonably good.